<a href="https://colab.research.google.com/github/MohammedYaseen97/idiomaticity-detection/blob/subtaskA_basic_exps/Mohammed_Yaseen_SemEval_2022_Task_2_Subtask_A_%5BBaseline%5D_%2B_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook provides a baseline for each setting in [Subtask A of SemEval 2022 Task 2](https://sites.google.com/view/semeval2022task2-idiomaticity#h.qq7eefmehqf9). In addition this provides some helpful pre-processing scripts that you are free to use with your experiments. 

Please start by stepping through this notebook so you have a clear idea as to what is expected of the task and what you need to submit. 

These baselines are based on the results described in the paper “[AStitchInLanguageModels: Dataset and Methods for the Exploration of Idiomaticity in Pre-Trained Language Models](https://arxiv.org/abs/2109.04413)”. 

## Zero-shot setting: Methodology 

Note that in the zero-shot setting you are NOT allowed to train the model using the one-shot data. 

In the zero-shot setting, we choose to include the context (the sentences preceding and succeeding the one containing the idioms). We do not add the idiom as an additional feature (in the “second input sentence”). This is based on the results presented in the dataset paper. 

We use Multilingual BERT for this setting.

## One-shot setting: Methodology

In the one shot setting, we train the model on both the zero-shot and one-shot data. In this setting, we exclude the context (the sentences preceding and succeeding the one containing the idioms) and also add the idiom as an additional feature in the “second sentence”. Again, this is based on the results presented in the dataset paper. 

We also use Multilingual BERT for this setting.


# Setup 

Download the Task data and evaluation scripts

In [ ]:
!git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git

Cloning into 'SemEval_2022_Task2-idiomaticity'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 123 (delta 48), reused 61 (delta 15), pack-reused 0
Receiving objects: 100% (123/123), 2.50 MiB | 8.78 MiB/s, done.
Resolving deltas: 100% (48/48), done.


Download the “AStitchInLanguageModels” code which we make use of. 

In [ ]:
!git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git

Cloning into 'AStitchInLanguageModels'...
remote: Enumerating objects: 1030, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 1030 (delta 11), reused 4 (delta 4), pack-reused 1013
Receiving objects: 100% (1030/1030), 79.59 MiB | 14.31 MiB/s, done.
Resolving deltas: 100% (394/394), done.


Download and install an editable version of huggingfaces transformers. 

In [ ]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers/
!pip install --editable .
%cd /content/ 

Cloning into 'transformers'...
remote: Enumerating objects: 101286, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 101286 (delta 114), reused 126 (delta 52), pack-reused 101018
Receiving objects: 100% (101286/101286), 94.91 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (74663/74663), done.
/content/transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 27.8 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 596 kB 52.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Running setup.py develop

Required for run_glue ... 

In [ ]:
## run_glue needs this. 
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 18.5 MB/s 
     |████████████████████████████████| 140 kB 65.8 MB/s 
     |████████████████████████████████| 212 kB 46.7 MB/s 
     |████████████████████████████████| 1.1 MB 66.8 MB/s 
     |████████████████████████████████| 127 kB 74.3 MB/s 
     |████████████████████████████████| 144 kB 53.4 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 271 kB 74.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Editable install requires runtime restart unless we do this. 

In [ ]:
import site
site.main()


# Imports and Helper functions

In [ ]:
import os
import csv

from pathlib import Path

In [ ]:
def load_csv( path, delimiter=',' ) : 
  header = None
  data   = list()
  with open( path, encoding='utf-8') as csvfile:
    reader = csv.reader( csvfile, delimiter=delimiter ) 
    for row in reader : 
      if header is None : 
        header = row
        continue
      data.append( row ) 
  return header, data


In [ ]:
def write_csv( data, location ) : 
  with open( location, 'w', encoding='utf-8') as csvfile:
    writer = csv.writer( csvfile ) 
    writer.writerows( data ) 
  print( "Wrote {}".format( location ) ) 
  return


# Pre-process: Create train and dev and evaluation data in required format

In the zero-shot setting, we choose to include the context (the sentences preceding and succeeding the one containing the idioms). We do not add the idiom as an additional feature (in the “second input sentence”). 

In the one shot setting, we train the model on both the zero-shot and one-shot data. In this setting, we exclude the context (the sentences preceding and succeeding the one containing the idioms) and also add the idiom as an additional feature in the “second sentence”. 


## Functions for pre-processing

### _get_train_data

This function generates training data in the format required by the huggingface’s example script. It will include and exclude the MWE and the context based on parameters. 


In [ ]:
def _get_train_data( data_location, file_name, include_context, include_idiom ) :
    
    file_name = os.path.join( data_location, file_name ) 

    header, data = load_csv( file_name )

    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]
        
    # ['DataID', 'Language', 'MWE', 'Setting', 'Previous', 'Target', 'Next', 'Label']
    out_data = list()
    for elem in data :
        label     = elem[ header.index( 'Label'  ) ]
        sentence1 = elem[ header.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ header.index( 'Previous' ) ], elem[ header.index( 'Target' ) ], elem[ header.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ header.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        out_data.append( this_row )
        assert len( out_header ) == len( this_row )
    return [ out_header ] + out_data

### _get_dev_eval_data

This function generates training dev and eval data in the format required by the huggingface’s example script. It will include and exclude the MWE and the context based on parameters. 

Additionally, if there is no gold label provides (as in the case of eval) it will generate a file that can be used to generate predictions.


In [ ]:
def _get_dev_eval_data( data_location, input_file_name, gold_file_name, include_context, include_idiom ) :

    input_headers, input_data = load_csv( os.path.join( data_location, input_file_name ) )
    gold_header  = gold_data = None
    if not gold_file_name is None : 
        gold_header  , gold_data  = load_csv( os.path.join( data_location, gold_file_name  ) )
        assert len( input_data ) == len( gold_data )

    # ['ID', 'Language', 'MWE', 'Previous', 'Target', 'Next']
    # ['ID', 'DataID', 'Language', 'Label']
    
    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]

    out_data = list()
    for index in range( len( input_data ) ) :
        label = 1
        if not gold_file_name is None : 
            this_input_id = input_data[ index ][ input_headers.index( 'ID' ) ]
            this_gold_id  = gold_data [ index ][ gold_header  .index( 'ID' ) ]
            assert this_input_id == this_gold_id
            
            label     = gold_data[ index ][ gold_header.index( 'Label'  ) ]
            
        elem      = input_data[ index ]
        sentence1 = elem[ input_headers.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ input_headers.index( 'Previous' ) ], elem[ input_headers.index( 'Target' ) ], elem[ input_headers.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ input_headers.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        assert len( out_header ) == len( this_row ) 
        out_data.append( this_row )
        

    return [ out_header ] + out_data


### create_data

This function generates the training, development and evaluation data. 


In [ ]:
"""
Based on the results presented in `AStitchInLanguageModels' we work with not including the idiom for the zero shot setting and including it in the one shot setting.
"""
def create_data( input_location, output_location, lang=None) :

    
    ## Zero shot data
    train_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv' if lang is None else 'train_zero_shot_'+lang.lower()+'.csv',
        include_context = True,
        include_idiom   = False
    )
    write_csv( train_data, os.path.join( output_location, 'ZeroShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv' if lang is None else 'dev_'+lang.lower()+'.csv',
        gold_file_name   = 'dev_gold.csv' if lang is None else 'dev_gold_'+lang.lower()+'.csv', 
        include_context  = True,
        include_idiom    = False
    )        
    write_csv( dev_data, os.path.join( output_location, 'ZeroShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv' if lang is None else 'eval_'+lang.lower()+'.csv',
        gold_file_name   = None , ## Don't have gold evaluation file -- submit to CodaLab
        include_context  = True,
        include_idiom    = False
    )
    write_csv( eval_data, os.path.join( output_location, 'ZeroShot', 'eval.csv' ) )


    ## OneShot Data (combine both for training)
    train_zero_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv' if lang is None else 'train_zero_shot_'+lang.lower()+'.csv',
        include_context = False,
        include_idiom   = True
    )
    train_one_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_one_shot.csv' if lang is None else 'train_one_shot_'+lang.lower()+'.csv',
        include_context = False,
        include_idiom   = True
    )

    assert train_zero_data[0] == train_one_data[0] ## Headers
    train_data = train_one_data + train_zero_data[1:]
    write_csv( train_data, os.path.join( output_location, 'OneShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv' if lang is None else 'dev_'+lang.lower()+'.csv',
        gold_file_name   = 'dev_gold.csv' if lang is None else 'dev_gold_'+lang.lower()+'.csv', 
        include_context  = False,
        include_idiom    = True
    )        
    write_csv( dev_data, os.path.join( output_location, 'OneShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv' if lang is None else 'eval_'+lang.lower()+'.csv',
        gold_file_name   = None,
        include_context  = False,
        include_idiom    = True
    )
    write_csv( eval_data, os.path.join( output_location, 'OneShot', 'eval.csv' ) )

    return

## Setup and Create data

In [ ]:
!ls 

AStitchInLanguageModels  SemEval_2022_Task2-idiomaticity
sample_data		 transformers


In [ ]:
def _get_lang( data_location, file_name, lang ) :
  # Filter csv files by language
    
  file_name = os.path.join( data_location, file_name ) 

  header, data = load_csv( file_name )

  # ['DataID', 'Language', 'MWE', 'Setting', 'Previous', 'Target', 'Next', 'Label']
  out_data = list()
  for elem in data :
      if elem[ header.index( 'Language'  ) ] == lang:
        out_data.append( elem )
  # write_csv( eval_data, os.path.join( data_location, file_name+"_"+lang.tolower()+".csv" ) )
  return [header] + out_data

In [ ]:
def create_lang_input(file_location, lang):
  train0_lang_data = _get_lang(
      data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
      file_name  = 'train_zero_shot.csv',
      lang=lang
  )
  write_csv( train0_lang_data, os.path.join( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', "train_zero_shot_"+lang.lower()+".csv" ) )

  train1_lang_data = _get_lang(
      data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
      file_name  = 'train_one_shot.csv',
      lang=lang
  )
  write_csv( train1_lang_data, os.path.join( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', "train_one_shot_"+lang.lower()+".csv" ) )

  dev_lang_data = _get_lang(
      data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
      file_name  = 'dev.csv',
      lang=lang
  )
  write_csv( dev_lang_data, os.path.join( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', "dev_"+lang.lower()+".csv" ) )

  dev_gold_lang_data = _get_lang(
      data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
      file_name  = 'dev_gold.csv',
      lang=lang
  )
  write_csv( dev_gold_lang_data, os.path.join( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', "dev_gold_"+lang.lower()+".csv" ) )

  eval_lang_data = _get_lang(
      data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
      file_name  = 'eval.csv',
      lang=lang
  )
  write_csv( eval_lang_data, os.path.join( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', "eval_"+lang.lower()+".csv" ) )

In [ ]:
# create_lang_input('SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', 'EN')

In [ ]:
outpath = 'Data'
    
Path( os.path.join( outpath, 'ZeroShot' ) ).mkdir(parents=True, exist_ok=True)
Path( os.path.join( outpath, 'OneShot' ) ).mkdir(parents=True, exist_ok=True)

create_data( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', outpath)

Wrote Data/ZeroShot/train.csv
Wrote Data/ZeroShot/dev.csv
Wrote Data/ZeroShot/eval.csv
Wrote Data/OneShot/train.csv
Wrote Data/OneShot/dev.csv
Wrote Data/OneShot/eval.csv


# Zero Shot Setting

## Baseline 1 : Monolingual BERT for zero-shot english idiomaticity detection 

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 15.0 MB/s 


In [ ]:
# English language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-uncased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

07/25/2022 22:57:00 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/25/2022 22:57:00 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_mod

## Baseline 2 : Multiingual BERT for zero-shot multilingual idiomaticity detection

In [ ]:
# Multi language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-multilingual-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

07/26/2022 07:27:16 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2022 07:27:16 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_mod

## Exp 01 : Monolingual adapter-based BERT for zero-shot english idiomaticity detection

In [ ]:
!pip install adapter-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.9 MB 32.7 MB/s 
     |████████████████████████████████| 880 kB 53.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4d99ed0910beb31440062d9c86a8b9175fb5a755324ae72a25f1764a552d694c
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
# English language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters_new.py \
  --model_name_or_path 'bert-base-uncased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 10.0 \
  --evaluation_strategy "epoch" \
  --output_dir models/ZeroShot/0/ \
  --overwrite_output_dir \
  --seed 0 \
  --train_file      Data/ZeroShot/train.csv \
  --validation_file Data/ZeroShot/dev.csv \
	--evaluation_strategy "epoch" \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

07/26/2022 02:44:06 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2022 02:44:06 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
labe

## Exp 02 : Multilingual adapter-based BERT for zero-shot multilingual idiomaticity detection

In [ ]:
# Multi language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters_new.py \
  --model_name_or_path 'bert-base-multilingual-cased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 10.0 \
  --evaluation_strategy "epoch" \
  --output_dir models/ZeroShot/0/ \
  --overwrite_output_dir \
  --seed 0 \
  --train_file      Data/ZeroShot/train.csv \
  --validation_file Data/ZeroShot/dev.csv \
	--evaluation_strategy "epoch" \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

07/26/2022 08:09:05 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2022 08:09:05 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
labe

## Exp 03 : Monolingual MirrorWiC BERT for zero-shot english idiomaticity detection

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 30.8 MB/s 


In [ ]:
# English language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'cambridgeltl/mirrorwic-bert-base-uncased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 15 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

07/26/2022 05:56:32 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2022 05:56:32 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_mod

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
## Create save path
!mkdir -p /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/0/
## Copy saved model.
!cp -r /content/models/ZeroShot/0/* /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/0/

In [ ]:
## Bring back saved model here. 
#!mkdir -p /content/models/ZeroShot/0/
# !cp -r /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/ZeroShot/0/* /content/models/ZeroShot/0/

# One Shot Setting

## Baseline 3 : Monolingual BERT for one-shot english idiomaticity detection

In [ ]:
# English language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-uncased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/OneShot/1/ \
    	--seed 1 \
    	--train_file      Data/OneShot/train.csv \
    	--validation_file Data/OneShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

07/26/2022 06:21:56 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2022 06:21:56 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_mod

## Baseline 4 : Multilingual BERT for one-shot multilingual idiomaticity detection 

In [ ]:
# Multi language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-multilingual-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/OneShot/1/ \
    	--seed 1 \
    	--train_file      Data/OneShot/train.csv \
    	--validation_file Data/OneShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

07/26/2022 07:46:50 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2022 07:46:50 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_mod

## Exp 04 : Monolingual adapter-based BERT for one-shot english idiomaticity detection

In [ ]:
!pip install adapter-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.9 MB 9.1 MB/s 
     |████████████████████████████████| 880 kB 55.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7f013bb2fbe2bf71bf76d6812eb60064f0bbbe7b9e122426451f070874801893
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
# English language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters_new.py \
  --model_name_or_path 'bert-base-uncased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 10.0 \
  --evaluation_strategy "epoch" \
  --output_dir models/OneShot/1/ \
  --seed 1 \
  --train_file      Data/OneShot/train.csv \
  --validation_file Data/OneShot/dev.csv \
  --overwrite_output_dir \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

07/26/2022 07:00:42 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2022 07:00:42 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
labe

## Exp 05 : Multilingual adapter-based BERT for one-shot multilingual idiomaticity detection

In [ ]:
# Multi language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters_new.py \
  --model_name_or_path 'bert-base-multilingual-cased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 10.0 \
  --evaluation_strategy "epoch" \
  --output_dir models/OneShot/1/ \
  --seed 1 \
  --train_file      Data/OneShot/train.csv \
  --validation_file Data/OneShot/dev.csv \
  --overwrite_output_dir \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

07/26/2022 08:23:54 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2022 08:23:54 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
labe

## Exp 06 : Monolingual MirrorWiC BERT for one-shot english idiomaticity detection

In [ ]:
# English language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'cambridgeltl/mirrorwic-bert-base-uncased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 15 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/OneShot/1/ \
    	--seed 1 \
    	--train_file      Data/OneShot/train.csv \
    	--validation_file Data/OneShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

07/26/2022 06:35:54 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2022 06:35:54 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_mod

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
## Create save path
!mkdir -p /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/OneShot/1/
## Copy saved model.
!cp -r /content/models/OneShot/1/* /content/gdrive/MyDrive/ColabData/SemEval2022Task2/TaskA/OneShot/1/